In [0]:


import numpy as np
import random
#FUNCTIONS TO MANIPULATE NESTED LISTS

#flatten nested list
def flatten(x):                   
  y=[]
  for t in x:
    y=y+t
  return y

#record positions in nested list. Returns list of tuples (t,k,j) t:type, k:word, j:positions within word 
def positions(string):             
  positions_string=[0]*len(string) 
  k=0
  while k<len(string):
    l=len(string[k])
    p_s=[0]*l
    j=0
    while j<l:     
      p_s[j]=(string[k][j],k,j)
      j=j+1
    positions_string[k]=p_s
    k=k+1
  return positions_string

#pop and top function of np.array
def np_pop(array):
     array=array.tolist()
     p=array.pop()
     array=np.array(array)
     return [array,p]

def np_top(array):
     array=array.tolist()
     array.reverse()
     array.pop()
     array.reverse()
     array=np.array(array)
     return array

#top function of list : removes first element 
def top(list1):
     list1.reverse()
     list1.pop()
     list1.reverse()
     return list1
     
#extracts a column from a nested list or list of tuples. returns a list
def column(list1,k):
     list1=np.array(list1)
     col_k = list1[:,k].tolist()
     return col_k






In [0]:
#PARSING FUNCTIONS
def right_inv(x,y):
     if y==x+'^r':
       return True 
     else:
       return False
     
def left_inv(x,y):
     if x==y+'^l':
       return True
     else:
       return False

def check_reduction(x,y):
     if left_inv(x,y)==True or right_inv(x,y)==True:
       return True
     else:
       return False 


def reduce(word1,wordt):     #lazy parsing 
         k=0
         word2=wordt[:]
         while k<len(wordt) and len(word1)>0:
            t_prec = word1[-1]
            t=word2[k]
            C=check_reduction(t_prec,t)
            if C==True:
               word1.pop()
               word2=top(word2)
               k=k+1
            else:
               k=k+1
            return word1+word2


#picks the left inverse of a given reduced type, from the reduction stack
def left_inverse(reductions,r):            
            red_flat=flatten(reductions)
            k=red_flat.index(r)
            left_inv=red_flat[k-1]
            return left_inv





#check if T is a critical type. If so it returns the matching reduction, else it returns T
def find_critical(right_reductions, T):
      if len(right_reductions)>0:
         right_reductions.reverse()
         for r in right_reductions:
             C=check_reduction(r[0],T[0])
             if C==False:
                if right_reductions.index(r)==len(right_reductions)-1:
                   return T
                else:
                   continue
             else:
                return r
                break
         right_reductions.reverse()  
      else:
         return T  


#resolves criticality: returns True if we accept the criticality, False if we ignore it 
def decide_critical(string,R,Rc,T):  
       flat_pos=flatten(positions(string))
       k=flat_pos.index(T)
       left_r= left_inverse(R,Rc)
       x=left_r[0]
       y=Rc[0]
       string_flat=flatten(string)
       string_k=string_flat[k:]
       c_x=[t for t in string_k if t==x]
       c_y=[t for t in string_k if t==y]
       count_x=len(c_x)+1
       count_y=len(c_y)
       if count_y>=count_x:
         
         return True
       else: 
         return False



#modifies reduction stack (in case of accepted criticality)
def amend_reduction(p0,R,R_stack, Rc, T):
                       left_r=left_inverse(R,Rc)
                       R_stack.remove(Rc)   
                       R_stack=R_stack+[T]
                       R.remove([left_r,Rc])
                       R=R+[[Rc,T]]
                       p0=p0+[left_r]            
                       return [p0,R,R_stack]

#tests parsing relative to a given type
def test(reductions,test_type, c_type):
      test=reduce(reductions,test_type)
      if test==[]:
        print('parsed correctly to type',c_type )
        
      else:
        print('NOT parsed to type',c_type, 'leftover types:', reductions)
      return reductions

#parsing algorithm 
            
def parse(dict, sentence):
     string=[list(random.choice(tuple(dict[x]))) for x in sentence] 
     print('sentence:',sentence)
     print('grammatical types:',string)
     print('parsing:')
     p0=[]
     R=[]
     R_stack=[]
     pos = positions(string)
     p0=p0+pos[0]    
     print('processed string:',sentence[0])
     k=1
     print('reductions:', [])
     print('reduced stack',column(p0,0))
     while k<len(pos):       
       p=pos[k]
       p1=p[:]
       print('processed string:',sentence[0:k+1])
       
       for T in p: 
           t=T[0]
           print('processed type:',T)
           if len(p0)>0:        
               t_prec=p0[-1][0]
               T_prec=p0[-1]
               C=check_reduction(t_prec,t) 
               if C==True:     
                  R_ij=[[T_prec,T]]  
                  p0.pop()
                  p1=top(p1)   
                  R=R+R_ij
                  print('reductions:', R)
                  print('reduced stack:',column(p0+p1,0))
                  R_stack=R_stack+[T]  
               else:      
                  Rc=find_critical(R_stack,T)
                  if Rc==T:
                     R_stack=[]
                     break
                  else:   
                    print(t,'is a critical type')
                    D=decide_critical(string,R,Rc,T)
                    if D==True:
                       A=amend_reduction(p0,R,R_stack, Rc, T)
                       p0,R,R_stack=A[0],A[1],A[2]                       
                       p1=top(p1)
                       print('reduced stack:',column(p0+p1,0))
                       print('reductions amended',R)
                       
                    else:
                       print('ignore criticality')
                       R_stack=[]
                       break
           else:
               Rc=find_critical(R_stack,T)
               if Rc==T:
                     R_stack=[]
                     break
               else:   
                    print(T, 'is a critical type') 
                    D=decide_critical(string,R,Rc,T)
                    if D==True:
                       A=amend_reduction(p0,R,R_stack, Rc, T)
                       p0,R,R_stack=A[0],A[1],A[2]
                       p1=top(p1)  
                       print('reduced stack:',column(p0+p1,0))
                       print('amended reductions:',R)
                    else:
                       print('ignore criticality')
                       R_stack=[]
                       break



       p0=p0+p1         
       print('reduced stack:',column(p0,0))
       k=k+1

     p=column(p0,0)
     Test=test(p,['s^r'], ['s'])
     print('Reductions:')
     return R  






In [30]:
dict={'alice':{('n')}, 'loves':{('n^r', 's', 'n^l')}, 'bob':{('n')}, 'very':{('n^r','n^l')},'much':{('n','n')}}
#dict can be ambiguous 
s= ['alice', 'loves', 'bob', 'very','much'] 
parse(dict,s)


sentence: ['alice', 'loves', 'bob', 'very', 'much']
grammatical types: [['n'], ['n^r', 's', 'n^l'], ['n'], ['n^r', 'n^l'], ['n', 'n']]
parsing:
processed string: alice
reductions: []
reduced stack ['n']
processed string: ['alice', 'loves']
processed type: ('n^r', 1, 0)
reductions: [[('n', 0, 0), ('n^r', 1, 0)]]
reduced stack: ['s', 'n^l']
processed type: ('s', 1, 1)
reduced stack: ['s', 'n^l']
processed string: ['alice', 'loves', 'bob']
processed type: ('n', 2, 0)
reductions: [[('n', 0, 0), ('n^r', 1, 0)], [('n^l', 1, 2), ('n', 2, 0)]]
reduced stack: ['s']
reduced stack: ['s']
processed string: ['alice', 'loves', 'bob', 'very']
processed type: ('n^r', 3, 0)
n^r is a critical type
reduced stack: ['s', 'n^l', 'n^l']
reductions amended [[('n', 0, 0), ('n^r', 1, 0)], [('n', 2, 0), ('n^r', 3, 0)]]
processed type: ('n^l', 3, 1)
reduced stack: ['s', 'n^l', 'n^l']
processed string: ['alice', 'loves', 'bob', 'very', 'much']
processed type: ('n', 4, 0)
reductions: [[('n', 0, 0), ('n^r', 1, 0)], 

[[('n', 0, 0), ('n^r', 1, 0)],
 [('n', 2, 0), ('n^r', 3, 0)],
 [('n^l', 3, 1), ('n', 4, 0)],
 [('n^l', 1, 2), ('n', 4, 1)]]